# Load module

In [1]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image
# import requests

# Load pre-trained ResNet50 

In [2]:
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)
model.eval()  # Set to evaluation mode

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\UNS_CT/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth


100.0%


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

# Preprocessing

In [4]:
preprocess = transforms.Compose([
    transforms.Resize(256),               # Resize shortest side to 256
    transforms.CenterCrop(224),           # Crop to 224x224 (ResNet50 input size)
    transforms.ToTensor(),                 # Convert to tensor and scale [0, 1]
    transforms.Normalize(                  # Imagenet mean/std normalization
        mean=[0.485, 0.456, 0.406], 
        std=[0.229, 0.224, 0.225]
    )
])

# Load and preprocess the image

In [5]:
image_path = r'D:\Programming_File\Tensorflow_Files\datasets\test_set\dog9002.jpg'
img = Image.open(image_path).convert("RGB")   # Ensure RGB mode
img_t = preprocess(img)
batch_t = img_t.unsqueeze(0)  # Add batch dimension

PIL.Image.Image

# Prediction

In [8]:
with torch.no_grad():
    outputs = model(batch_t)
    probabilities = torch.nn.functional.softmax(outputs[0], dim=0)

# Decode predictions (similar to decode_predictions)

In [9]:
import json
from urllib.request import urlopen

LABELS_URL = "https://raw.githubusercontent.com/pytorch/hub/master/imagenet_classes.txt"
imagenet_classes = [line.strip() for line in urlopen(LABELS_URL)]

In [10]:
# Get top-5 predictions
top5_prob, top5_catid = torch.topk(probabilities, 5)
for i in range(top5_prob.size(0)):
    print(f"{imagenet_classes[top5_catid[i]]}: {top5_prob[i].item():.4f}")

b'Labrador retriever': 0.8708
b'flat-coated retriever': 0.0442
b'German short-haired pointer': 0.0152
b'black-and-tan coonhound': 0.0091
b'pug': 0.0085
